In [1]:
from fastai import *
from fastai.text import *
import pandas as pd
import os
from pathlib import Path

In [4]:
path = Path('/home/louis/Documents/NLP_satl/data')

In [5]:
path.ls()

[PosixPath('/home/louis/Documents/NLP_satl/data/train_full.tsv'),
 PosixPath('/home/louis/Documents/NLP_satl/data/.gitignore'),
 PosixPath('/home/louis/Documents/NLP_satl/data/info.pkl'),
 PosixPath('/home/louis/Documents/NLP_satl/data/twitter-datasets.zip'),
 PosixPath('/home/louis/Documents/NLP_satl/data/sample_submission.csv'),
 PosixPath('/home/louis/Documents/NLP_satl/data/twitter-datasets'),
 PosixPath('/home/louis/Documents/NLP_satl/data/test_data.txt')]

In [25]:
with open(path/'train_neg_full.txt', 'r') as f :
    neg_tweets = f.readlines()

In [26]:
with open(path/'train_pos_full.txt', 'r') as f :
    pos_tweets = f.readlines()

In [27]:
len(neg_tweets), len(pos_tweets)

(1250000, 1250000)

In [28]:
df = pd.DataFrame({'tweet' : neg_tweets + pos_tweets, 'label' : ['neg' for i in range(len(neg_tweets))] + ['pos' for i in range(len(pos_tweets))]})

In [29]:
df.head()

,tweet,label
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,neg
1,glad i dot have taks tomorrow ! ! #thankful #s...,neg
2,1-3 vs celtics in the regular season = were fu...,neg
3,<user> i could actually kill that girl i'm so ...,neg
4,<user> <user> <user> i find that very hard to ...,neg


In [31]:
df.shape

(2500000, 2)

In [32]:
df.to_csv(path/'tweets.csv')

In [34]:
#data_lm = TextLMDataBunch.from_csv(path, 'tweets.csv')

In [35]:
#data_lm.save('data_lm_export.pkl')

In [3]:
data_lm = load_data(path,'data_lm_export.pkl' )

In [13]:
data_lm.show_batch()

idx,text
0,"lmaooo ! okay ! advertise that twittaaa anywhere / anytime \n xxbos a plump and perky turkey ( paperback plump and perky turkey is a marshall cavendish publication . < url > \n xxbos < user > yeah , you guys should it was fun \n xxbos < user > xxunk woof * grabs a handful of ur bottom * u'll do for me \n xxbos"
1,"> do n't you think sex was enough ? no need to introduce architecture in the same conversation . just go and have that drink . \n xxbos < user > today is the day \n xxbos i love waking up next to this boy < url > \n xxbos we all have targets , # usain bolts is to be a living legend "" a living"
2,"xxunk \n xxbos < user > nooo .. i want him so bad tho .. like he just so chill and sweet \n xxbos < user > i 'm waiting on a recipe for honey bun cake \n xxbos turner clc utah utes notepads , 5 x 8 inches , 2 packs ( 8 xxunk the official collegiate 2 - pack notepads is the p ... <"
3,"kill me soon "" < user > mosquitoes have killed more humans than all the wars in history . "" \n xxbos < user > who said i liked u ? \n xxbos girlfriend , boyfriend , friend everythinq has an end except for family # xxunk ! \n xxbos < user > thanks lmao ! i tried \n xxbos rey go to the hospital \n"
4,", she tryna jerk meh \n xxbos < user > tu xxunk out of twitter ! ... have a nice day ! \n xxbos "" batman is n't afraid to beat up a girl , he beats up cat girl ! "" "" ... i play cat woman on batman arkham city "" "" but i would n't beat you up "" \n xxbos like this shit"


In [14]:
#data_clas = TextClasDataBunch.from_csv(path, 'tweets.csv', vocab=data_lm.train_ds.vocab, bs=32, text_cols='tweet', label_cols='label')

In [15]:
data_clas.save('data_clas_export.pkl')

In [ ]:
DataBunch.save()

In [16]:
data_clas.show_batch()

text,target
xxbos party at your house ? bap bap ! lml rt < user > my birthday is in 10 days ! ! ! \ 355 \ 240 \ 274 \ 355 \ 276 \ 211 \ 355 \ 240 \ 274 \ 355 \ 276 \ 210 \ 355 \ 240 \ 274 \ 355 \ 276 \ 211 \ 355 \ 240 \ 274 \ 355 \ 276 \,pos
"xxbos # < - < ) i was gunna give you this waffle , ( > - ) > # but then i was like , ( > # ' < ) i m hungry ( > - ' < ) so i ate it \n",neg
xxbos how school works : 2 + 2 = 4 . homework : 2 + 4 + 2 = 8 . exam : john had 4 apples . he eats one and gives one to a friend . calculate the suns mass ... \n,neg
xxbos microstar int . xxunk - v2 2 gb memory ram kit ( 2x1 gb ) ( a - tech brand 2 gb ddr - 266 ( pc2100 ) kit ( 2x1 gb ) for micros ... < url > \n,neg
xxbos when u hope for a hot guy to sit next to u on a plane ... and u get creepy xxunk ! ! then hot guys smiles and gives u the ' i 'm sorry ' face ! \n,neg


In [5]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [6]:
learn.layer_groups

[Sequential(
   (0): WeightDropout(
     (module): LSTM(400, 1150, batch_first=True)
   )
   (1): RNNDropout()
 ), Sequential(
   (0): WeightDropout(
     (module): LSTM(1150, 1150, batch_first=True)
   )
   (1): RNNDropout()
 ), Sequential(
   (0): WeightDropout(
     (module): LSTM(1150, 400, batch_first=True)
   )
   (1): RNNDropout()
 ), Sequential(
   (0): Embedding(60007, 400, padding_idx=1)
   (1): EmbeddingDropout(
     (emb): Embedding(60007, 400, padding_idx=1)
   )
   (2): LinearDecoder(
     (decoder): Linear(in_features=400, out_features=60007, bias=True)
     (output_dp): RNNDropout()
   )
 )]

In [7]:
learn.layer_groups[0]

Sequential(
  (0): WeightDropout(
    (module): LSTM(400, 1150, batch_first=True)
  )
  (1): RNNDropout()
)

In [19]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.117996,3.913267,0.357143,28:27


In [20]:
learn.predict("This is a review about", n_words=10)

'This is a review about rumors behind our winning everyones night ! round ... cap'

In [21]:
learn.save_encoder('ft_enc')

In [22]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.522422,0.443533,0.783238,13:31


In [24]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))


epoch,train_loss,valid_loss,accuracy,time
0,0.377730,0.318243,0.859916,16:18


In [25]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))



epoch,train_loss,valid_loss,accuracy,time
0,0.318376,0.287228,0.874658,36:28


In [36]:
learn.predict('I am happy')

(Category pos, tensor(1), tensor([0.0527, 0.9473]))

In [27]:
learn.save('first_fastai')

In [33]:
learn.data.show_batch()

text,target
xxbos party at your house ? bap bap ! lml rt < user > my birthday is in 10 days ! ! ! \ 355 \ 240 \ 274 \ 355 \ 276 \ 211 \ 355 \ 240 \ 274 \ 355 \ 276 \ 210 \ 355 \ 240 \ 274 \ 355 \ 276 \ 211 \ 355 \ 240 \ 274 \ 355 \ 276 \,pos
xxbos b . o . b . so hard to breathe ( itunes b . o . b . so hard to breathe ( itunes ) download : b . o . b . so hard to breathe ... < url > \n,neg
"xxbos * sips acid * "" < user > < user > sorry oh ... please ... is that u in ur avatar ? i hear it is . if so , do n't u value or rate urself ? "" \n",neg
xxbos microstar int . xxunk combo - fr ( ms xxunk - 010 ) 1 gb memory ram upgrade ( a - tech brand 1 gb ddr 2 - 533 ( pc2 - 4200 ) for mi ... < url > \n,neg
xxbos 4.5 k tweet gives a s / o to my < user > ! i miss you so much it 's crazy . ca n't wait for you to come back ! hope you 're havin fun there tho ! \n,pos


In [37]:
with open(path/'test_data.txt', 'r') as f:
    test_data = f.readlines()

In [38]:
test_data[:10]

['1,sea doo pro sea scooter ( sports with the portable sea-doo seascootersave air , stay longer in the water and ... <url>\n',
 "2,<user> shucks well i work all week so now i can't come cheer you on ! oh and put those batteries in your calculator ! ! !\n",
 '3,i cant stay away from bug thats my baby\n',
 "4,<user> no ma'am ! ! ! lol im perfectly fine and not contagious anymore lmao\n",
 '5,whenever i fall asleep watching the tv , i always wake up with a headache\n',
 "6,<user> he needs to get rid of that thing ! it scares me lol but he don't need a car either . he needs drivers ed again .\n",
 '7,its whatever . in a terrible mood ( (\n',
 "8,yesss ! rt <user> <user> thanks jordan , i love you and i'm gonna call you later !\n",
 '9,my friend <user> text me to check up on me last night .\n',
 '10,<user> #followback please . when will ur #unitytour come to europe and sweden ? ?\n']

In [45]:
learn.predict(test_data[0].split(',', 1)[1])[1].item()

0

In [46]:
splitted_test_data = [(eval(s.split(',', 1)[0]), s.split(',', 1)[1], -1 if learn.predict(s.split(',', 1)[1])[1].item()==0 else 1) for s in test_data]

In [48]:
df_test_pred = pd.DataFrame({'Id' : [r[0] for r in splitted_test_data], 'Prediction' : [r[2] for r in splitted_test_data]})

In [49]:
df_test_pred.head()

,Id,Prediction
0,1,-1
1,2,-1
2,3,-1
3,4,1
4,5,-1


In [51]:
df_test_pred.to_csv('first_fastai_try.csv', index=False)

In [52]:
learn.save(path/'first_fastai.pkl')

# With preprocessed tweets

In [12]:
df = pd.read_csv(path/'train_full.tsv', sep='\t', header=0, error_bad_lines=False)

b'Skipping line 760410: expected 2 fields, saw 3\n'
b'Skipping line 790940: expected 2 fields, saw 3\n'


In [14]:
n_tweets = df.shape[0]
permutation = np.random.permutation(n_tweets)
pctg = 0.2
train_idxs = permutation[int(pctg*n_tweets):]
valid_idxs = permutation[:int(pctg*n_tweets)]
train_df = df.iloc[train_idxs]
valid_df = df.iloc[valid_idxs]

In [15]:
n_tweets, train_df.shape[0], valid_df.shape[0]

(2458295, 1966636, 491659)

In [16]:
#data_lm = TextLMDataBunch.from_df(path, train_df, valid_df)
#data_lm.save('data_lm_preproc_fastai.pkl')

In [17]:
data_lm = load_data(path,'data_lm_preproc_fastai.pkl' )

In [18]:
data_lm.show_batch()

idx,text
0,"< number > likes . < repeat > girl with boobs falling out : "" lol i just sneezed "" - < number > likes . < sadface > xxbos < user > because he is a christian . xxbos < user > what about you anything exciting planned ? x xxbos < user > < number > yrs this makes me feel so old , remember wen it came"
1,"no filling , xxunk xxbos say hi to tish please ? ( < user > live on < url > xxbos everytime i see something about < hashtag > # xxunk it makes me tear up xxbos < user > that 's very kind of you . btw i hope mrs yg is making good progress . ) ) < hashtag > # xxunk xxbos amo < elong > xxunk"
2,meet her soon xxbos girls that always wanna go through your phone - . - obviously i ' m going to delete what should n't be there xxbos rt < user > i could go for a xxunk right now ! < hashtag > # xxunk but really xxbos < user > loool why am i missing this xxbos < user > hahaha ! hope your not talking about me
3,xxbos < user > haha wel < elong > maybe not eh but there is has to be like there 's so many people out there i ' m just one fea :} haha xxbos xxunk xxunk jiten = < smile > ictionary of today 's design ( japanese edition < url > xxbos good morning tweeps ! have a great wednesday ahead . be real and be safe god
4,"meant ! you ' ve forgotten me i see how it is xxbos < hashtag > # yougetmajorpointsif you have ambition , goals , and are going somewhere with your life . but still have fun . aye xxbos pretty close to my baby < url > xxbos best night with my bestie for lifee do n't know what i would do with out < user > < hashtag >"


In [19]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [20]:
learn.predict("This is a review about", n_words=10)

'This is a review about Kid Sixty and Brandon , dispatched to'

In [26]:
learn.fit_one_cycle(1, 0.01)

epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 